<a href="https://colab.research.google.com/github/Srinivaskolli45/-EVA8-Assignment/blob/main/S4/assignment4_step_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms    

  

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #input 1x28x28   ==> Output 10x28x28  | RF 3
        self.conv1 = nn.Sequential(nn.Conv2d(1, 10, 3,padding=1),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      
        #input 10x28x28 ==> Output 10x28x28 | RF 5
        self.conv2 = nn.Sequential(nn.Conv2d(10, 10, 3,padding=1),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
      
        #input 10x28x28 ==> Output 10x14x14 | RF  6
        self.transitionblk1 = nn.MaxPool2d(2,2)

        #input 10x14x14 ==> Output 32x12x12 | RF 10
        self.conv3 = nn.Sequential(nn.Conv2d(10,32,3),
                                      nn.BatchNorm2d(32),
                                      nn.ReLU())
                                     

        #input 32x12x12 ==> Output 10x10x10 | RF 14
        self.conv4 = nn.Sequential(nn.Conv2d(32,10,3),
                                      nn.BatchNorm2d(10),
                                      nn.ReLU())
                                      
         
                                      
        #input 10x10x10 ==> OUtput 20x8x8 | RF 18
        self.conv5 = nn.Sequential(nn.Conv2d(10,20,3),
                                      nn.BatchNorm2d(20),
                                      nn.ReLU())
                                      
        #input 20x8x8 ==> OUtput 10x8x8 | RF 20
        self.conv6 = nn.Sequential(nn.Conv2d(20,10,1),
                                    nn.BatchNorm2d(10),
                                     nn.ReLU())
                                     
       
        #input 10x5x5 ==> OUtput 10x1x1 | RF 33
        self.avgPoolblk = nn.AvgPool2d(6,6)
    def forward(self, x):
        x = self.conv1(x)  #input 1x28x28   ==> Output 10x28x28  | RF 3
        x = self.conv2(x)  #input 10x28x28   ==> Output 10x28x28  | RF 5      
        x = self.transitionblk1(x) #input 10x28x28 ==> Output 10x14x14 | RF  6
        x = self.conv3(x)  #input 10x14x14 ==> Output 32x12x12 | RF 10
        x = self.conv4(x)   #input 32x12x12 ==> Output 10x10x10 | RF 14
        x = self.conv5(x)   #input 10x10x10 ==> Output 20x8x8 | RF 18
        x = self.conv6(x)        #input 20x8x8 ==> Output 10x8x8 | RF 18
        x = self.avgPoolblk(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)  


In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model,input_size=(1,28,28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
              ReLU-3           [-1, 10, 28, 28]               0
            Conv2d-4           [-1, 10, 28, 28]             910
       BatchNorm2d-5           [-1, 10, 28, 28]              20
              ReLU-6           [-1, 10, 28, 28]               0
         MaxPool2d-7           [-1, 10, 14, 14]               0
            Conv2d-8           [-1, 32, 12, 12]           2,912
       BatchNorm2d-9           [-1, 32, 12, 12]              64
             ReLU-10           [-1, 32, 12, 12]               0
           Conv2d-11           [-1, 10, 10, 10]           2,890
      BatchNorm2d-12           [-1, 10, 10, 10]              20
    

<ipython-input-6-49fca3dfaf6a>:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
import torchvision


torch.manual_seed(1)
batch_size = 128

#kwargs = {'num_workers': 4, 'pin_memory': True} if use_cuda else {}
kwargs = dict(num_workers= 4, pin_memory= True if use_cuda else False,shuffle=True,batch_size=batch_size)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,)),
                        transforms.RandomRotation(degrees=7)
                    ])),**kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #transforms.RandomRotation(degrees=7)
                    ])),**kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    train_loss = 0
    correct = 0
    processed=0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        train_loss +=loss
        pred = output.argmax(dim=1,keepdim=True)
        
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accurary={100*correct/processed:0.2f}')

    
   # print('\nTrain set: average loss: {:.4f}, accuracy: {}/{} ({:.0f}%)\n'.format(
   
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
from torch.optim.lr_scheduler import StepLR
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
schedular = StepLR(optimizer=optimizer,step_size=6,gamma=0.1)
for epoch in range(15):
    print(f'Epoch--> {epoch+1}')
    train(model, device, train_loader, optimizer, epoch)
    print("Current learning rate:", optimizer.param_groups[0]['lr'])
    schedular.step()
    test(model, device, test_loader)
     

Epoch--> 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-6-49fca3dfaf6a>:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.12884025275707245 batch_id=468 Accurary=94.61: 100%|██████████| 469/469 [00:28<00:00, 16.19it/s]

Current learning rate: 0.1



Test set: Average loss: 0.1032, Accuracy: 9693/10000 (96.93%)

Epoch--> 2


loss=0.08339408785104752 batch_id=468 Accurary=98.12: 100%|██████████| 469/469 [00:30<00:00, 15.14it/s]


...

loss=0.00481827836483717 batch_id=468 Accurary=99.43: 100%|██████████| 469/469 [00:31<00:00, 14.99it/s]


Current learning rate: 0.0010000000000000002

Test set: Average loss: 0.0203, Accuracy: 9944/10000 (99.44%)

Epoch--> 15


loss=0.02203756757080555 batch_id=468 Accurary=99.44: 100%|██████████| 469/469 [00:29<00:00, 15.73it/s]

Current learning rate: 0.0010000000000000002



Test set: Average loss: 0.0200, Accuracy: 9943/10000 (99.43%)

